## **200968108 Sec-A 27**
## **Week-3** Problem Solving Agents & Search.

### **The Game :**
According to the “Six Degrees of Kevin Bacon” game, anyone in the Hollywood film
industry can be connected to Kevin Bacon within six steps, where each step onsists of finding a film that two actors both starred in. To solve the problem, find the shortest path between any two actors by choosing a sequence of movies that connects them. For example, the shortest path between Jennifer Lawrence and Tom Hanks is 2:

Jennifer Lawrence is connected to Kevin Bacon by both starring in “X-Men: First
Class,” and Kevin Bacon is connected to Tom Hanks by both starring in “Apollo 13.”

**Problem Solving Agent:**

Given two actors nodes in the graph we need to find the distance (shortest path)
between the nodes.

Write a python program to determine how many “degrees of separation” apart two
actors are. Find the distance or the degree of separation., using

a. Breadth first search

b. Depth first search

**Distribution Code:**

Data & Download the distribution code from:

https://cdn.cs50.net/ai/2020/x/projects/0/degrees.zip

The distribution code contains two sets of CSV data files: one set in the large directory and
one set in the small directory. Use the small dataset for ease of testing and
experimentation. Each dataset consists of three CSV files.

1. small/people.csv: each person has a unique id, corresponding with their id in
IMDb’s database. They also have a name, and a birth year.

2. small/movies.csv: You’ll see here that each movie also has a unique id, in addition
to a title and the year in which the movie was released.

3. small/stars.csv: This file establishes a relationship between the people in
people.csv and the movies in movies.csv. Each row is a pair of a person_id value
and movie_id value.

For example: The first row (ignoring the header), for example, states that the person with id 102 starred in the movie with id 104257. Checking that against people.csv and movies.csv, you’ll find that this line is saying that Kevin Bacon starred in the movie “A Few Good Men.”

**4.degrees.py:**

At the top, several data structures are defined to store information from the CSV
files. The names dictionary is a way to look up a person by their name: it maps names to a set of corresponding ids (because it’s possible that multiple actors have the same name). The people dictionary maps each person’s id to another dictionary with values
for the person’s name, birth year, and the set of all the movies they have starred in.

And the movies dictionary maps each movie’s id to another dictionary with values for that movie’s title, release year, and the set of all the movie’s stars. The load_data function loads data from the CSV files into these data structures.

The main function in this program first loads data into memory (the directory from
which the data is loaded can be specified by a command-line argument). Then, the
function prompts the user to type in two names. The person_id_for_name function
retrieves the id for any person (and handles prompting the user to clarify, in the event that multiple people have the same name). The function then calls the shortest_path function to compute the shortest path between the two people, and prints out the path.

**The shortest_path function has to be coded using**

**a.Breadth First Search**

**b.Depth First Search**

*(Below is the code given in **utils.py**)*

In [55]:
class Node():
    def __init__(self, state, parent, action):
        self.state = state
        self.parent = parent
        self.action = action

In [56]:
class StackFrontier():
    def __init__(self):
        self.frontier = []

    def add(self, node):
        self.frontier.append(node)

    def contains_state(self, state):
        return any(node.state == state for node in self.frontier)

    def empty(self):
        return len(self.frontier) == 0

    def remove(self):
        if self.empty():
            raise Exception("empty frontier")
        else:
            node = self.frontier[-1]
            self.frontier = self.frontier[:-1]
            return node

In [57]:
class QueueFrontier(StackFrontier):

    def remove(self):
        if self.empty():
            raise Exception("empty frontier")
        else:
            node = self.frontier[0]
            self.frontier = self.frontier[1:]
            return node

*(Below is the code given in **degrees.py**)*

In [58]:
import csv
import sys

# uncomment the below line if you want to run in an editor
# from util import Node, StackFrontier, QueueFrontier

# Maps names to a set of corresponding person_ids
names = {}

# Maps person_ids to a dictionary of: name, birth, movies (a set of movie_ids)
people = {}

# Maps movie_ids to a dictionary of: title, year, stars (a set of person_ids)
movies = {}

In [59]:
def load_data(directory):
    """
    Load data from CSV files into memory.
    """
    # Load people
    with open(f"{directory}/people.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            people[row["id"]] = {
                "name": row["name"],
                "birth": row["birth"],
                "movies": set()
            }
            if row["name"].lower() not in names:
                names[row["name"].lower()] = {row["id"]}
            else:
                names[row["name"].lower()].add(row["id"])

    # Load movies
    with open(f"{directory}/movies.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            movies[row["id"]] = {
                "title": row["title"],
                "year": row["year"],
                "stars": set()
            }

    # Load stars
    with open(f"{directory}/stars.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            try:
                people[row["person_id"]]["movies"].add(row["movie_id"])
                movies[row["movie_id"]]["stars"].add(row["person_id"])
            except KeyError:
                pass

In [60]:
def main():
    
    directory = sys.argv[1] if len(sys.argv) == 2 else "/content/drive/MyDrive/Colab Notebooks/Sem 6 Labs/degrees/large"

    # Load data from files into memory
    print("Loading data...")
    load_data(directory)
    print("\nData loaded.\n")

    source = person_id_for_name(input("Name: "))
    if source is None:
        sys.exit("Person not found.")
    target = person_id_for_name(input("Name: "))
    if target is None:
        sys.exit("Person not found.")

    path = shortest_path(source, target)

    if path is None:
        print("Not connected.")
    else:
        degrees = len(path)
        print(f"\n{degrees} degrees of separation.\n")
        path = [(None, source)] + path
        for i in range(degrees):
            person1 = people[path[i][1]]["name"]
            person2 = people[path[i + 1][1]]["name"]
            movie = movies[path[i + 1][0]]["title"]
            print(f"{i + 1}: {person1} and {person2} starred in {movie}")

In [61]:
def shortest_path(source, target):
    """
    Returns the shortest list of (movie_id, person_id) pairs
    that connect the source to the target.

    If no possible path, returns None.
    """
  
    count_explored = 0
    print(f"\nSearching: {count_explored}", end="", flush=True)

    # Initiate starting node and frontier
    start = Node(state=source, parent=None, action=None)
    
    opt = int(input("\nDo you want to use DFS/BFS?: \n 1. BFS\n 2.DFS\n"))

    if(opt == 1):
      frontier = QueueFrontier()
    elif(opt == 2):
      frontier = StackFrontier()


    frontier.add(start)

    explored = set()

    while True:
        if frontier.empty(): # If frontier is empty, then no solution
            print()
            return None
        
        # Checkout node from frontier
        node = frontier.remove()

        # Update count
        count_explored += 1
        print(f"\rSearching: {count_explored}", end="", flush=True)            

        # Mark node as explored
        explored.add(node.state)

        # Add neighbors to frontier
        for action, state in neighbors_for_person(node.state):
            if not frontier.contains_state(state) and state not in explored:
                child = Node(state=state, parent=node, action=action)

                # If child is the target, get solution
                if child.state == target:
                    node = child
                    solution = []

                    # Loop backwards to find solution
                    while node.parent is not None:
                        solution.append((node.action, node.state))
                        node = node.parent

                    # print newline
                    print()

                    # return reversed list
                    return solution[::-1]
                else:
                    frontier.add(child)

    # TODO
    raise NotImplementedError

In [62]:
def person_id_for_name(name):
    """
    Returns the IMDB id for a person's name,
    resolving ambiguities as needed.
    """
    person_ids = list(names.get(name.lower(), set()))
    if len(person_ids) == 0:
        return None
    elif len(person_ids) > 1:
        print(f"Which '{name}'?")
        for person_id in person_ids:
            person = people[person_id]
            name = person["name"]
            birth = person["birth"]
            print(f"ID: {person_id}, Name: {name}, Birth: {birth}")
        try:
            person_id = input("\nIntended Person ID: ")
            if person_id in person_ids:
                return person_id
        except ValueError:
            pass
        return None
    else:
        return person_ids[0]

In [63]:
def neighbors_for_person(person_id):
    """
    Returns (movie_id, person_id) pairs for people
    who starred with a given person.
    """
    movie_ids = people[person_id]["movies"]
    neighbors = set()
    for movie_id in movie_ids:
        for person_id in movies[movie_id]["stars"]:
            neighbors.add((movie_id, person_id))
    return neighbors

#### ***Finding the shortest path using BFS :***


In [75]:
'''
  checking degree of separation for - William Holden & Kevin Bacon(ID: 102)
'''
# calling the main() function to execute the code
main()

Loading data...

Data loaded.

Name: William Holden
Name: Kevin Bacon
Which 'Kevin Bacon'?
ID: 102, Name: Kevin Bacon, Birth: 1958
ID: 9323132, Name: Kevin Bacon, Birth: 

Intended Person ID: 102

Searching: 0
Do you want to use DFS/BFS?: 
 1. BFS
 2.DFS
1
Searching: 81

3 degrees of separation.

1: William Holden and Robert Duvall starred in Network
2: Robert Duvall and Robert De Niro starred in The Godfather: Part II
3: Robert De Niro and Kevin Bacon starred in Sleepers


In [84]:
'''
  checking degree of separation for - Cary Elwes & Colin Firth
'''
# calling the main() function to execute the code
main()

Loading data...

Data loaded.

Name: Cary Elwes
Name: Colin Firth

Searching: 0
Do you want to use DFS/BFS?: 
 1. BFS
 2.DFS
1
Searching: 14

2 degrees of separation.

1: Cary Elwes and Mary Steenburgen starred in Wish You Were Dead
2: Mary Steenburgen and Colin Firth starred in Hope Springs


#### ***Finding the shortest path using DFS :***


In [77]:
'''
  checking degree of separation for - William Holden & Kevin Bacon(ID: 102)
'''
# calling the main() function to execute the code
main()

Loading data...

Data loaded.

Name: William Holden
Name: Kevin Bacon
Which 'Kevin Bacon'?
ID: 102, Name: Kevin Bacon, Birth: 1958
ID: 9323132, Name: Kevin Bacon, Birth: 

Intended Person ID: 102

Searching: 0
Do you want to use DFS/BFS?: 
 1. BFS
 2.DFS
2
Searching: 20

15 degrees of separation.

1: William Holden and Ernest Borgnine starred in The Revengers
2: Ernest Borgnine and Anne De Salvo starred in Spike of Bensonhurst
3: Anne De Salvo and Loryn Locklin starred in Taking Care of Business
4: Loryn Locklin and Clifton Collins Jr. starred in Fortress
5: Clifton Collins Jr. and Kevin McNally starred in Painted Beauty
6: Kevin McNally and Emily Bruni starred in King Lear: Live from Shakespeare's Globe
7: Emily Bruni and Brad Gorton starred in The Case
8: Brad Gorton and Stacy Hart starred in Get Real
9: Stacy Hart and Charlie Bond starred in Dead Air
10: Charlie Bond and Georgia Annable starred in Hellriser
11: Georgia Annable and Adam Collins starred in Essex Heist
12: Adam Collins

In [85]:
'''
  checking degree of separation for - Cary Elwes & Colin Firth
'''
# calling the main() function to execute the code
main()

Loading data...

Data loaded.

Name: Cary Elwes
Name: Colin Firth

Searching: 0
Do you want to use DFS/BFS?: 
 1. BFS
 2.DFS
2
Searching: 20

16 degrees of separation.

1: Cary Elwes and Michael Biehn starred in Psych:9
2: Michael Biehn and Sean Patrick Flanery starred in The Insatiable
3: Sean Patrick Flanery and Michael Paré starred in Raging Angels
4: Michael Paré and John Fallon starred in The Shelter
5: John Fallon and Claudia Jurt starred in Deaden
6: Claudia Jurt and Erica Anderson starred in Little Death
7: Erica Anderson and Cindel Chartrand starred in Maz
8: Cindel Chartrand and Alexandra Woodward starred in Misbehaviour
9: Alexandra Woodward and Neal McDonough starred in Silent Men
10: Neal McDonough and Amy Smart starred in Bad Country
11: Amy Smart and David Cubitt starred in No Clue
12: David Cubitt and Chandra West starred in A Perfect Son
13: Chandra West and Tom Cavanagh starred in Something More
14: Tom Cavanagh and David Mazouz starred in The Games Maker
15: David Ma

#### **Observation:**
* From the above output, we can observe that the Degree of Separation is always higher in case of DFS search(more than 15 and sometimes more than 50) compared to BFS search(always less than or equal to 6).
* This indicates the high time and space complexity in DFS than BFS.
* We can conclude that BFS search works better and is always more preferable over DFS search.